In [1]:
import numpy as np 
import pandas as pd 
import requests 
import xlsxwriter 
import math

In [2]:
stocks = pd.read_csv('sp_500_stocks.csv')
from secrets import IEX_CLOUD_API_TOKEN

In [3]:
symbol = 'AAPL'
api_url = f'https://sandbox.iexapis.com/stable/stock/{symbol}/stats?token={IEX_CLOUD_API_TOKEN}'
data = requests.get(api_url).json()
data

{'companyName': 'Apple Inc',
 'marketcap': 2487926127036,
 'week52high': 154.77,
 'week52low': 90.35,
 'week52highSplitAdjustOnly': 149.29,
 'week52lowSplitAdjustOnly': 90.11,
 'week52change': 0,
 'sharesOutstanding': 16924246526,
 'float': 0,
 'avg10Volume': 101192288,
 'avg30Volume': 82621440,
 'day200MovingAvg': 131.47,
 'day50MovingAvg': 133.89,
 'employees': 147590,
 'ttmEPS': 4.6,
 'ttmDividendRate': 0.8454887140337198,
 'dividendYield': 0.005823616077137066,
 'nextDividendDate': '',
 'exDividendDate': '2021-05-05',
 'nextEarningsDate': '2021-07-16',
 'peRatio': 31.85985225347749,
 'beta': 1.6064616352245298,
 'maxChangePercent': 54.91038795711888,
 'year5ChangePercent': 5.58262972503914,
 'year2ChangePercent': 1.977922222491978,
 'year1ChangePercent': 0.5594231196016197,
 'ytdChangePercent': 0.10350985010016733,
 'month6ChangePercent': 0.11719637612610773,
 'month3ChangePercent': 0.0864019997453569,
 'month1ChangePercent': 0.1446239690649806,
 'day30ChangePercent': 0.15012436512

In [4]:
data['year1ChangePercent']

0.5594231196016197

In [5]:
# Function sourced from 
# https://stackoverflow.com/questions/312443/how-do-you-split-a-list-into-evenly-sized-chunks
def chunks(lst, n):
    """Yield successive n-sized chunks from lst."""
    for i in range(0, len(lst), n):
        yield lst[i:i + n]   
        
symbol_groups = list(chunks(stocks['Ticker'], 100))
symbol_strings = []
for i in range(0, len(symbol_groups)):
    symbol_strings.append(','.join(symbol_groups[i]))
#     print(symbol_strings[i])

my_columns = ['Ticker', 'Price', 'One-Year Price Return', 'Number of Shares to Buy']

In [6]:

final_dataframe = pd.DataFrame(columns = my_columns)

for symbol_string in symbol_strings:
#     print(symbol_strings)
    batch_api_call_url = f'https://sandbox.iexapis.com/stable/stock/market/batch/?types=stats,quote&symbols={symbol_string}&token={IEX_CLOUD_API_TOKEN}'
    data = requests.get(batch_api_call_url).json()
    for symbol in symbol_string.split(','):
        final_dataframe = final_dataframe.append(
                                        pd.Series([symbol, 
                                                   data[symbol]['quote']['latestPrice'],
                                                   data[symbol]['stats']['year1ChangePercent'],
                                                   'N/A'
                                                   ], 
                                                  index = my_columns), 
                                        ignore_index = True)
        
    
final_dataframe

,Ticker,Price,One-Year Price Return,Number of Shares to Buy
0,A,155.682,0.698193,N/A
1,AAL,21.015,0.739164,N/A
2,AAP,218.230,0.564209,N/A
3,AAPL,151.160,0.551168,N/A
4,ABBV,122.320,0.267702,N/A
...,...,...,...,...
500,YUM,117.700,0.365536,N/A
501,ZBH,161.580,0.317636,N/A
502,ZBRA,540.970,1.120998,N/A
503,ZION,52.170,0.629428,N/A


In [7]:
final_dataframe.sort_values('One-Year Price Return', ascending = False, inplace = True)
final_dataframe = final_dataframe[:51]
final_dataframe.reset_index(drop = True, inplace = True)
final_dataframe

,Ticker,Price,One-Year Price Return,Number of Shares to Buy
0,LB,77.400,3.554111,N/A
1,TPR,43.620,2.317566,N/A
2,DVN,28.604,2.066792,N/A
3,GPS,31.970,1.917709,N/A
4,FCX,35.980,1.834764,N/A
5,SIVB,582.370,1.765492,N/A
6,DFS,128.755,1.713939,N/A
7,COF,158.460,1.660315,N/A
8,DXC,41.372,1.624989,N/A
9,KSS,53.320,1.622042,N/A


In [8]:
def portfolio_input():
    global portfolio_size
    portfolio_size = input("Enter the value of your portfolio:")

    try:
        val = float(portfolio_size)
    except ValueError:
        print("That's not a number! \n Try again:")
        portfolio_size = input("Enter the value of your portfolio:")

portfolio_input()
print(portfolio_size)

Enter the value of your portfolio:10000000
10000000


In [9]:
position_size = float(portfolio_size) / len(final_dataframe.index)
for i in range(0, len(final_dataframe['Ticker'])):
    final_dataframe.loc[i, 'Number of Shares to Buy'] = math.floor(position_size / final_dataframe['Price'][i])
final_dataframe

C:\Users\91742\anaconda3\lib\site-packages\pandas\core\indexing.py:1720: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)


,Ticker,Price,One-Year Price Return,Number of Shares to Buy
0,LB,77.400,3.554111,2533
1,TPR,43.620,2.317566,4495
2,DVN,28.604,2.066792,6854
3,GPS,31.970,1.917709,6133
4,FCX,35.980,1.834764,5449
5,SIVB,582.370,1.765492,336
6,DFS,128.755,1.713939,1522
7,COF,158.460,1.660315,1237
8,DXC,41.372,1.624989,4739
9,KSS,53.320,1.622042,3677
